# **3. Группировка данных в DataFrame**

✍ Одна из основных задач анализа данных — это группировка данных и сравнение показателей в группах. Например, нам необходимо сравнить средний уровень заработной платы в зависимости от пола/уровня образования. Или же мы хотим проследить, какая группа клиентов приносит нам наибольший доход, чтобы направить своё внимание на эту группу.
В некоторых случаях группировки может быть достаточно, чтобы ответить на вопросы бизнеса. В других случаях это может стать первым шагом в более сложном анализе. Так, например, благодаря группировке мы можем выявлять признаки, которые не несут статистической значимости, или признаки, которые вносят наибольший вклад. 
Так или иначе, владение группировкой — важный навык, который открывает новые возможности по работе с данными.


МЕТОД GROUPBY()

В библиотеке Pandas для группировки данных по одному или нескольким признакам можно использовать метод groupby().

Основные параметры метода groupby()
•	by — имя или список имён столбцов, по которым производится группировка.
•	axis — ось, по которой производится группировка (0 — строки, 1 — столбцы). По умолчанию группировка производится по строкам.
•	as_index — добавляется ли дополнительный индекс к таблице. По умолчанию установлен на True.

Метод groupby() возвращает объект DataFrameGroupBy, который хранит в себе информацию о том, какие строки относятся к определённой группе, и сам по себе не представляет для нас интереса. 
Однако к этому объекту можно применять уже знакомые нам агрегирующие методы (mean, median, sum и т. д.), чтобы рассчитывать показатели внутри каждой группы.

Посмотрим на общую схему работы метода groupby() в Pandas:
 Источник изображения https://www.slideserve.com/stian/python-lab-pandas-iii 
Сначала мы разделяем данные на группы с помощью метода groupby(), после чего к каждой группе применяем агрегацию и объединяем результаты в новую таблицу.


ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ

Рассмотрим группировку данных на примере нашей таблицы с недвижимостью.
Применим агрегирующую функцию среднего к результату работы groupby(). В качестве столбца для группировки возьмём столбец типа объекта недвижимости (Type):


In [ ]:
melb_df.groupby(by='Type').mean(numeric_only=True)

Мы получили таблицу, на пересечении строк и столбцов которой находятся средние значения каждого числового признака в наших данных.
Обратите внимание на структуру получившейся таблицы: теперь на месте индексов стоят значения типа объекта недвижимости Type (house, townhouse, unit).
Примечание. Если мы хотим видеть тип объекта в качестве отдельного столбца таблицы, мы можем выставить параметр as_index на False:
 
Как правило, нам не нужна информация обо всех столбцах, поэтому агрегирующие методы можно применять только к интересующему нас столбцу. Например, давайте сравним средние цены на объекты в зависимости от их типа:


In [ ]:
melb_df.groupby('Type')['Price'].mean()

*Примечание*. Обратите внимание, что, так как мы считаем только один показатель (среднее) для одного столбца, в результате мы получаем объект Series.
Из этой маленькой таблицы видно, что наибольшей средней ценой обладают объекты типа house (дома, коттеджи, виллы). Следовательно, можно сделать вывод, что тип постройки является значимым фактором при определении цены объекта недвижимости.
Теперь давайте выясним, какие регионы (Regionname) наиболее удалены от центра Мельбурна.
Для этого найдём минимальное значение расстояния от центра города до объекта в зависимости от его региона. Результат отсортируем по убыванию расстояния:
Итак, наиболее удалёнными являются все регионы Victoria.


In [ ]:
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ

Чтобы рассчитать несколько агрегирующих методов, можно воспользоваться методом agg(), который принимает список строк с названиями агрегаций.
Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale). Результат отсортируем по количеству продаж в порядке убывания:


In [ ]:
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)


*Примечание*. Обратите внимание, что, так как мы считаем несколько показателей для одного столбца, в результате мы получаем объект DataFrame.
В результате применения метода agg(), в который мы передали список с названиями интересующих нас агрегирующих функций, мы получаем DataFrame со столбцами count, mean и max, где для каждого месяца рассчитаны соответствующие параметры. Результат сортируем по столбцу count.


*Какие интересные выводы можно сделать из этой таблицы?*

1	Пик продаж приходится на период весна-лето.

2	Средняя цена продаваемых объектов относительно стабильна и находится в пределах 1 млн. австралийских долларов с небольшими отклонениями (около 100 тыс. влево и вправо).

3	Прослеживается некоторая зависимость между сезоном и максимальной ценой объектов: в месяцы с большим спросом на объекты недвижимости цена также имеет наибольшие показатели. Можно сделать предположение, что это связано с повышением цен на элитные дома в периоды большого спроса.

*Примечание*. Если вам нужна полная информация обо всех основных статистических характеристиках внутри каждой группы, вы можете воспользоваться методом agg(), передав в качестве его параметра строку 'describe':


In [ ]:
melb_df.groupby('MonthSale')['Price'].agg('describe')

После базовых математических функций наиболее частым агрегированием является подсчёт числа уникальных значений. Так, например, мы можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять, в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода agg() строку 'nunique'. 
Более того, метод agg() поддерживает использование и других функций. Передадим дополнительно встроенную функцию set, чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов:


In [ ]:
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

Как и ожидалось, наименьшая конкуренция в наиболее удалённом регионе Western Victoria, а наибольшая — в центральном районе Northern Metropolitan.
✍ Методы группировки по одному признаку позволяют посмотреть на наши данные «в разрезе» группирующего признака и понять их взаимосвязь. Далее мы рассмотрим, как оценивать взаимосвязь между большим количеством признаков.
Предлагаем вам самим потренироваться в использовании группировки ↓


*Задание 3.1*

Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.
(7)

In [ ]:
import numpy as np

In [1]:
import pandas as pd

In [2]:
melb_data = pd.read_csv('data/melb_data_fe.csv')
melb_df = melb_data.copy()
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False)

Rooms
7     1.920700e+06
5     1.870260e+06
6     1.849366e+06
8     1.602750e+06
4     1.445282e+06
3     1.076081e+06
10    9.000000e+05
2     7.750812e+05
1     4.338245e+05
Name: Price, dtype: float64

*Задание 3.2*

Какой регион имеет наименьшее стандартное отклонение по географической широте (Lattitude)?
В качестве ответа запишите название этого региона
(Western Victoria)

In [3]:
melb_data = pd.read_csv('data/melb_data_fe.csv')
melb_df = melb_data.copy()
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

Regionname
Western Victoria              0.011579
Southern Metropolitan         0.043080
Eastern Metropolitan          0.047890
Northern Metropolitan         0.049639
Western Metropolitan          0.051251
South-Eastern Metropolitan    0.073411
Northern Victoria             0.084455
Eastern Victoria              0.147067
Name: Lattitude, dtype: float64

*Задание 3.3*

Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?

Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период.
*Подсказка *(1 из 3): Произведите фильтрацию по условию попадания данных о продажах в интервал указанных дат.
*Подсказка* (2 из 3): Сгруппируйте данные по признаку SellerG и посчитайте сумму продаж для каждой компании.
*Подсказка *(3 из 3): Отсортируйте данные по возрастанию.

(LITTLE)


In [4]:
melb_data = pd.read_csv('data/melb_data_fe.csv')
melb_df = melb_data.copy()
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
date1 = pd.to_datetime('2017-05-01')
date2 = pd.to_datetime('2017-09-01')
mask = (date1 <= melb_df['Date']) & (melb_df['Date']<= date2)
melb_df[mask].groupby('SellerG')['Price'].sum().sort_values(ascending=True)


SellerG
LITTLE             2742000.0
Cayzer             4439000.0
Burnham            4550500.0
Moonee             7328000.0
Thomson            8332000.0
Bells              8656000.0
Alexkarbon        10985000.0
McDonald          14637500.0
Rendina           15422276.0
Nick              16890000.0
Douglas           18341000.0
Buckingham        19033000.0
C21               19515000.0
Eview             19791500.0
Collins           20217000.0
Philip            22051800.0
Chisholm          23225000.0
Williams          23297000.0
Love              23365500.0
Purplebricks      23401000.0
O'Brien           23855508.0
HAR               25568000.0
Village           26473000.0
RW                29261000.0
Raine             30687700.0
Stockdale         35409800.0
Sweeney           36882750.0
Gary              39138400.0
Hodges            43231000.0
YPA               46354350.0
Miles             47582000.0
Kay               48569500.0
RT                50498000.0
Brad              55955000.0
Jas   